## Extract Properties and Sparse Matrices from every FOV

Import packages

In [ ]:
import glob
import h5py
from skimage.measure import label, regionprops
from matplotlib import pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
import cv2

Define sorting scripts

In [ ]:
def custom_sorth5(name_fov): 
    [other,value]=name_fov.split('sub')
    [value,other,other]=value.split('_')    
    return value
def custom_sortdapi(name_dapi): 
    [other,value]=name_dapi.split('sub')
    [value,other]=value.split('.')    
    return value

Read .h5 and .tif file and sort the list

In [ ]:
#Read all h5 files in a directory. 
h5_files=glob.glob('*.h5')
h5_files.sort(key=custom_sorth5)
#print(h5_files)

dapi_files=glob.glob('*.tif')
dapi_files.sort(key=custom_sortdapi)
#Get all the FOVs and arrange them in the position
#To get the range we have to add one
number_fovs=np.arange(1,len(h5_files)+1)
#fov_matrix=np.array(number_fovs).reshape(rows,columns)
fov_matrix=np.array(number_fovs).reshape(23,25)

dapi_fov_counter=0
all_fov_properties=[]
all_fov_sparses=[]

Iterate over the FOVs and extract sparse arrays

In [ ]:
for current_fov in h5_files:    
    coord_list=[]
    area_list=[]
    fov_list=[]
    #The h5 extension  is 'exported_data'
    #name_fov=h5_files[0]
    name_fov=current_fov
    fov = h5py.File(name_fov, 'r')
    mask=fov['/exported_watershed_masks'][:]
    mask_reduced=np.squeeze(mask, axis=2)

    #Get DAPI fov 
    dapi_fov= cv2.imread(dapi_files[dapi_fov_counter],cv2.IMREAD_GRAYSCALE)
    
    #Check which position the FOV occupies within the big scan
    #Position of FOV ilastik_masks_watershed1.h5
    [other,value]=name_fov.split('sub')
    [value,other,other]=value.split('_')
    (x,y)=np.where(fov_matrix==int(value))
    
    
   
    
    #Label the  mask
    mask_label=label(mask_reduced)
    plt.imshow(mask_reduced)
    type(mask)
    [height,width]=mask_reduced.shape
    

    #fig, ax = plt.subplots(figsize=(10, 6))
    #ax.imshow(mask_label)
    sparse_in_fov=np.zeros([1024,1024,np.amax(mask_label)])
    z_dimension=0
    for i in range(0,np.amax(mask_label)+1):
        xmask,ymask=np.where(mask_label==i)
        single_cell_mask=np.zeros([1024,1024])
        sparse_in_fov= np.insert(sparse_in_fov, z_dimension, single_cell_mask, axis=2)
        z_dimension= z_dimension+1
    #Regionprops. Extract properties from  each nuclei
    #Save all sparse array in the fov
    all_fov_sparses.append(sparse_in_fov)
    for region in regionprops(mask_label):
        # take regions with large enough are4as
        if region.area >  10:
            single_sparse=csr_matrix(region)
            # draw rectangle around segmented coins
            coord_cen = region.centroid
            coord_list.append((coord_cen))
            area_list.append(region.area)
            #minr, minc, maxr, maxc = region.bbox
            #rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                     # fill=False, edgecolor='red', linewidth=2)
            fov_list.append((int(x),int(y)))
            #ax.add_patch(rect)
    #ax.set_axis_off()
    #plt.tight_layout()
    #plt.show()
    single_fov_properties={"Fov_Position":fov_list,"Centroids":coord_list,"Area":area_list,"Sparse":sparse_in_fov}
    all_fov_properties[current_fov]=single_fov_properties
    
    
    #Move to the next dapi fov
    dapi_fov_counter=dapi_fov_counter+1

Save the data

In [ ]:
np.savez('fov_data.npz',all_fov_properties=all_fov_properties,)
